# Mass with Friction Simulation (1D) using RK-4 and Plotly

## **1. Setup and Imports**
We import `numpy` for numerical operations and `plotly` for plotting.

---

In [1]:
import numpy as np
import plotly.graph_objects as go

## **2. Parameters and ODE Definition**

In [2]:
mass = 1.0         # Mass (kg)
friction = 0.1     # Friction coefficient (kg/s)
dt = 0.01          # Time step (s)
total_time = 10.0  # Total simulation time (s)
n_steps = int(total_time / dt)  # Number of steps

def sensor_eq(state, noise_std=1.0):
    return state + np.random.normal(0, noise_std, state.shape) #Sensor with Gaussian Noise

def equations(t, state, control_force=0.0):
    x, v = state
    dxdt = v  # Velocity is the derivative of position
    dvdt = (control_force - friction * v) / mass  # Acceleration with friction and control force
    new_state = np.array([dxdt, dvdt]) 
    return new_state

## **3. RK-4 Method Implementation**
This function advances the state using the classical fourth-order Runge-Kutta method.

---

In [3]:
def rk4_step(func, t, state, dt, control_force = 0.0):
    k1 = func(t, state, control_force)
    k2 = func(t + dt/2, state + dt/2 * k1, control_force)
    k3 = func(t + dt/2, state + dt/2 * k2, control_force)
    k4 = func(t + dt, state + dt * k3, control_force)
    return state + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)

## **4. Running the Simulation**
We store the results and use RK-4 to update the state over time.

---

In [4]:
positions = []
velocities = []
times = np.linspace(0, total_time, n_steps)

state = np.array([0.0, 10.0])  # Initial position and velocity
for t in times:
    masurement = sensor_eq(state)
    positions.append(masurement[0])
    velocities.append(masurement[1])
    state = rk4_step(equations, t, state, dt)

## **5. Plotting the Results with Plotly**
The simulation results are plotted using `Plotly` to visualize the position and velocity over time.

---

In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=positions, mode='lines', name='Position'))
fig.add_trace(go.Scatter(x=times, y=velocities, mode='lines', name='Velocity'))

fig.update_layout(
    title='Mass with Friction Simulation (RK-4)',
    xaxis_title='Time (s)',
    yaxis_title='Value',
    legend_title='Variables'
)
fig.show()

## **PD Controller Implementation**
This function computes a proportional-derivative (PD) control force to guide the mass toward a desired position.

---

In [6]:
def pd_control(target_position, current_position, current_velocity, kp=2.0, kd=1.0):
    error = target_position - current_position
    derivative = -current_velocity
    control_force = kp * error + kd * derivative
    return control_force

## **Running the Simulation with PD Control**
We store the results and use RK-4 to update the state over time with PD control.

---

In [7]:
positions = []
velocities = []
times = np.linspace(0, total_time, n_steps)
target_position = 5.0  # Desired position

state = np.array([0.0, 10.0])  # Initial position and velocity
for t in times:
    measurement = sensor_eq(state)
    control_force = pd_control(target_position, measurement[0], measurement[1])
    positions.append(measurement[0])
    velocities.append(measurement[1])
    state = rk4_step(equations, t, state, dt, control_force)

fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=positions, mode='lines', name='Position'))
fig.add_trace(go.Scatter(x=times, y=velocities, mode='lines', name='Velocity'))
fig.add_trace(go.Scatter(x=times, y=[target_position]*len(times), mode='lines', name='Target Position', line=dict(dash='dash')))

fig.update_layout(
    title='Mass with Friction Simulation with PD Control (RK-4)',
    xaxis_title='Time (s)',
    yaxis_title='Value',
    legend_title='Variables'
)
fig.show()


# **Parameter identification**
We import `numpy` for numerical operations, `plotly` for plotting, and `least_squares` from `scipy.optimize` for parameter estimation.

---

In [8]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

### **Parameters and ODE Definition**

In [9]:
mass = 1.0         # Mass (kg)
true_friction = 0.5     # Friction coefficient (kg/s)
dt = 0.05          # Time step (s)
total_time = 20.0  # Total simulation time (s)
n_steps = int(total_time / dt)  # Number of steps

def sensor_eq(state, noise_std=1.0):
    return state + np.random.normal(0, noise_std, state.shape) #Sensor with Gaussian Noise

def equations(t, state, friction, control_force=0.0):
    x, v = state
    dxdt = v  # Velocity is the derivative of position
    dvdt = (control_force - friction * v) / mass  # Acceleration with friction and control force
    return np.array([dxdt, dvdt])

def rk4_step(func, t, state, dt, friction, control_force):
    k1 = func(t, state, friction, control_force)
    k2 = func(t + dt/2, state + dt/2 * k1, friction, control_force)
    k3 = func(t + dt/2, state + dt/2 * k2, friction, control_force)
    k4 = func(t + dt, state + dt * k3, friction, control_force)
    return state + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)

### **Simulating System Response**
We simulate the system response using the true friction coefficient.

---

In [10]:
positions = []
velocities = []
times = np.linspace(0, total_time, n_steps)

state = np.array([1.0, 5.0])  # Initial position and velocity
for t in times:
    measurement = sensor_eq(state)
    positions.append(measurement[0])
    velocities.append(measurement[1])
    state = rk4_step(equations, t, state, dt, true_friction, 0.0)

positions = np.array(positions)
velocities = np.array(velocities)

### **Least Squares Estimation of Friction Coefficient**
We use the least squares algorithm to estimate the friction coefficient by minimizing the difference between the simulated and actual positions.

---

In [11]:
def cost_function(friction_guess):
    state = np.array([1.0, 5.0])  # Initial conditions
    estimated_positions = []

    for t in times:
        estimated_positions.append(state[0])
        state = rk4_step(equations, t, state, dt, friction_guess[0], 0.0)

    squared_error = np.sum((np.array(estimated_positions) - positions) ** 2)
    return squared_error  # Sum of squared errors (SSE)

initial_guess = np.array([0.5])  # Initial friction guess
bounds = [(0.0, 10.0)]  # Friction must be positive

result = minimize(cost_function, initial_guess, method='trust-constr', bounds=bounds)
estimated_friction = result.x[0]

print(f"Estimated Friction Coefficient: {estimated_friction:.4f}")

Estimated Friction Coefficient: 0.5000


### **Simulating System Response with Estimated Friction**

In [12]:
estimated_positions = []
state = np.array([1.0, 5.0])
for t in times:
    estimated_positions.append(state[0])
    state = rk4_step(equations, t, state, dt, estimated_friction, 0.0)
estimated_positions = np.array(estimated_positions)

### **Plotting the Results**
The estimated response is compared with the true system response to validate the estimated friction coefficient.

---

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=positions, mode='lines', name='True System Response'))
fig.add_trace(go.Scatter(x=times, y=estimated_positions, mode='lines', name='Estimated Response', line=dict(dash='dash')))

fig.update_layout(
    title='Friction Estimation using Interior-Point Optimization',
    xaxis_title='Time (s)',
    yaxis_title='Position (m)',
    legend_title='Legend'
)
fig.show()

## **Full-State Feedback Control**
This function computes a full-state feedback control force using a state-feedback gain matrix `K` to regulate both position and velocity.

---

In [14]:
def full_state_feedback_control(target_state, current_state, K=np.array([10.0, 1.0])):
    error = target_state - current_state
    control_force = K @ error  # State feedback control law
    return control_force

## **Running the Simulation with Full-State Feedback Control and Noisy Sensor Readings**
We store the results and use RK-4 to update the state over time with full-state feedback control while incorporating sensor noise.

---

In [15]:
positions = []
velocities = []
times = np.linspace(0, total_time, n_steps)
target_state = np.array([0.0, 0.0])  # Desired position and velocity

state = np.array([2.0, 10.0])  # Initial position and velocity
for t in times:
    noisy_state = sensor_eq(state)
    control_force = full_state_feedback_control(target_state, noisy_state)
    positions.append(noisy_state[0])
    velocities.append(noisy_state[1])
    state = rk4_step(equations, t, state, dt, true_friction, control_force) 

fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=positions, mode='lines', name='Position'))
fig.add_trace(go.Scatter(x=times, y=velocities, mode='lines', name='Velocity'))
fig.add_trace(go.Scatter(x=times, y=[target_state[0]]*len(times), mode='lines', name='Target Position', line=dict(dash='dash')))

fig.update_layout(
    title='Mass with Friction Simulation with Full-State Feedback Control (RK-4)',
    xaxis_title='Time (s)',
    yaxis_title='Value',
    legend_title='Variables'
)
fig.show()

## **Kalman Filter Implementation**
This function implements a simple Kalman filter to estimate the system state based on noisy measurements.

---

In [16]:
# Kalman filter function
def kalman_filter(state_est, control_force, P_est, measurement, A, B, C, Q, R):
    # Prediction step
    state_pred = A @ state_est + B @ control_force  # (2x2)@(2x1) + (2x1)@(1x1) -> (2x1)
    P_pred = A @ P_est @ A.T + Q  # (2x2)@(2x2)@(2x2) + (2x2) -> (2x2)

    # Measurement update step
    K = P_pred @ C.T @ np.linalg.inv(C @ P_pred @ C.T + R)  # (2x2)@(2x1)@(1x1) -> (2x1)
    state_est = state_pred + K @ (measurement - C @ state_pred)  # (2x1) + (2x1) -> (2x1)
    P_est = (np.eye(2) - K @ C) @ P_pred  # (2x2)@(2x2) -> (2x2)

    return state_est, P_est

## **Running the Simulation with Kalman Filter and Full-State Feedback Control**
We store the results and use RK-4 to update the state over time with Kalman filtering for state estimation and full-state feedback control.

---

In [27]:
positions = []
velocities = []
times = np.linspace(0, total_time, n_steps)
target_state = np.array([5.0, 0.0])  # Desired position and velocity

# Kalman filter initialization
P_est = np.eye(2)  # Initial state covariance (2x2)
A = np.array([[1, dt], [0, 1 - dt/mass]])  # State transition matrix (2x2)
B = np.array([[0], [dt/mass]])  # Control matrix (2x1)
Q = np.array([[0.01, 0], [0, 0.01]])  # Process noise covariance (2x2)
R = np.array([[1.3]])  # Measurement noise covariance (1x1)
C = np.array([[1.0, 0.0]])  # Observation matrix (1x2)
control_force = np.array([[0.0]])  # Control force (1x1)

# Initialization
control_force = np.array([[0.0]])  # Ensure it's a 1x1 matrix
state_est_history = []  # Stores estimated states
measurement_history = []  # Stores measurements
state_est = np.array([[8.0], [0]])  # Initial state (2x1)

for t in times:
    measurement = sensor_eq(state)[0]  # Simulated noisy measurement
    state_est, P_est = kalman_filter(state_est, control_force, P_est, measurement, A, B, C, Q, R)

    control_force = np.array([[full_state_feedback_control(target_state, state_est.flatten())]])  # Keep it 1x1

    state_est_history.append(state_est.flatten())  # Store full state estimate (as 1D array)
    measurement_history.append(measurement.flatten())  # Store measurement (as 1D array)

    state = rk4_step(equations, t, state, dt, true_friction, control_force.item())  # Fix applied here

# Convert lists to numpy arrays for easier plotting
state_est_history = np.array(state_est_history)
measurement_history = np.array(measurement_history)

# Plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=[target_state[0]]*len(times), mode='lines', name='Target Position', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=times, y=state_est_history[:, 0], mode='lines', name='State Estimate (Position)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=times, y=state_est_history[:, 1], mode='lines', name='State Estimate (Velocity)', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=times, y=measurement_history[:, 0], mode='lines', name='Measurements', marker=dict(color='red', size=5)))

fig.update_layout(
    title='Mass with Friction Simulation with Kalman Filter and Full-State Feedback Control (RK-4)',
    xaxis_title='Time (s)',
    yaxis_title='Value',
    legend_title='Variables'
)

fig.show()